Initialization

In [2]:
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
import plotly.express as px

Data Cleaning

In [110]:
#load dataset
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Student Performance Prediction/student_performance.csv')
df

,StudentID,Name,Gender,AttendanceRate,StudyHoursPerWeek,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study Hours,Attendance (%),Online Classes Taken
0,1.0,John,Male,85.0,15.0,78.0,1.0,High,80.0,4.8,59.0,False
1,2.0,Sarah,Female,90.0,20.0,85.0,2.0,Medium,87.0,2.2,70.0,True
2,3.0,Alex,Male,78.0,10.0,65.0,0.0,Low,68.0,4.6,92.0,False
3,4.0,Michael,Male,92.0,25.0,90.0,3.0,High,92.0,2.9,96.0,False
4,5.0,Emma,Female,NaN,18.0,82.0,2.0,Medium,85.0,4.1,97.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,Kenneth Murray,Male,85.0,20.0,NaN,1.0,High,72.0,0.8,80.0,True
996,4497.0,Amy Stout,Female,91.0,NaN,86.0,0.0,High,90.0,3.9,80.0,True
997,1886.0,NaN,Male,85.0,8.0,82.0,2.0,Low,68.0,0.4,54.0,False
998,7636.0,Joseph Sherman,Male,88.0,17.0,60.0,2.0,High,85.0,0.9,53.0,True


In [ ]:
#finding which column has NA values, and the sum of NA in each column
df.info()
df.isna().sum()

#StudentID has null values, primary key cannot have null values, replace with number increments
df = df.sort_values('StudentID')
df['StudentID'] = range(1, len(df) + 1)

#Handling null values in 'Gender'
#if a gender is heavily skewed, assign the gender to the most frequent
#else remove those records so it does not heavily affect visualization and prediction
gender_dist = df['Gender'].value_counts()
df = df.dropna(subset=['Gender'])

#Handling null in numerical columns
#finding number of nulls in these columns 
#attendance - 39, studyhours (daily) - 23, previousgrade - 33, extracurricular - 42
attend_avg = int(df['AttendanceRate'].mean())
attend_dist = pd.DataFrame()
attend_dist['count'] = df['AttendanceRate'].value_counts()
attend_dist['percent'] = round(attend_dist['count'] / attend_dist['count'].sum() * 100, 2)
print(attend_dist)                      #exponential difference between the highest and others 
df['AttendanceRate'] = df['AttendanceRate'].fillna(attend_avg)
df['AttendanceRate'] = df['AttendanceRate'].astype(int)

df = df[df['Study Hours'] >=0]          #drop negative study hours
df = df.dropna(subset=['Study Hours'])  #drop NA because distributions have minimal difference esp with float
                                        #and not many NA

prev_grade_dist = pd.DataFrame()
prev_grade_dist['count'] = df['PreviousGrade'].value_counts()
prev_grade_dist['percent'] = round(prev_grade_dist['count'] / prev_grade_dist['count'].sum() * 100, 2)
print(prev_grade_dist)
df = df.dropna(subset=['PreviousGrade']) #drop NA because distributions have minimal difference, and not many NA
df['PreviousGrade'] = df['PreviousGrade'].astype(int)

curricular_dist = pd.DataFrame()
curricular_dist['count'] = df['ExtracurricularActivities'].value_counts()
curricular_dist['percent'] = round(curricular_dist['count'] / curricular_dist['count'].sum() * 100, 2)
print(curricular_dist)                  #1 and 2 shows a big difference to 0 and 3
np.random.seed(42)
mask = df['ExtracurricularActivities'].isna()
df.loc[mask, 'ExtracurricularActivities'] = np.random.choice([1, 2], size=mask.sum())
df['ExtracurricularActivities'] = df['ExtracurricularActivities'].astype(int)

#Handling null values in 'ParentalSupport'
support_dist = df['ParentalSupport'].value_counts()
print(support_dist)             #drop NA because distributions have minimal difference, and not many NA
df = df.dropna(subset=['ParentalSupport'])

#Handling null values and standardize 'Online Classes Taken' column
online_dist = df['Online Classes Taken'].value_counts()
print(online_dist)             #drop NA because distributions have minimal difference, and not many NA
df = df.dropna(subset=['Online Classes Taken'])
#Yes and No are more meaningful to TRUE and FALSE for EDA labels, and does not affect prediction 
df['Online Classes Taken'] = df['Online Classes Taken'].astype(bool)
df['Online Classes Taken'] = df['Online Classes Taken'].map({True: 'Yes', False: 'No'})

#Check for duplicates
df.duplicated().sum()
student = df['Name'].value_counts().reset_index(name='count')
student.loc[student['count'] == 2]  #3 same student names
df.loc[df['Name']=='Andrea Frey']   
df.loc[df['Name']=='Anthony Smith']
df.loc[df['Name']=='Erica Miller']  #assume same student names but different student 
                                    #since there is difference in multiple columns

#Drop unnecessary columns
col_drop = ['StudyHoursPerWeek','Attendance (%)']
col_drop = [col for col in col_drop if col in df.columns]
df = df.drop(columns=col_drop)
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   StudentID                  960 non-null    float64
 1   Name                       966 non-null    object 
 2   Gender                     952 non-null    object 
 3   AttendanceRate             960 non-null    float64
 4   StudyHoursPerWeek          950 non-null    float64
 5   PreviousGrade              967 non-null    float64
 6   ExtracurricularActivities  957 non-null    float64
 7   ParentalSupport            978 non-null    object 
 8   FinalGrade                 960 non-null    float64
 9   Study Hours                976 non-null    float64
 10  Attendance (%)             959 non-null    float64
 11  Online Classes Taken       975 non-null    object 
dtypes: float64(8), object(4)
memory usage: 93.9+ KB
                count  percent
AttendanceRate            

,StudentID,Name,Gender,AttendanceRate,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,Study Hours,Online Classes Taken
0,1,John,Male,85,78,1,High,80.0,4.8,No
1,2,Sarah,Female,90,85,2,Medium,87.0,2.2,Yes
2,3,Alex,Male,78,65,0,Low,68.0,4.6,No
3,4,Michael,Male,92,90,3,High,92.0,2.9,No
4,5,Emma,Female,85,82,2,Medium,85.0,4.1,Yes
...,...,...,...,...,...,...,...,...,...,...
888,995,Jordan Thornton,Female,78,85,1,Low,72.0,1.2,Yes
918,996,Matthew Winters,Male,78,60,0,Medium,72.0,2.8,No
975,997,Nancy Gibson,Female,78,82,1,Medium,90.0,2.2,Yes
978,998,Nina Fowler,Male,70,78,3,Medium,68.0,4.1,Yes


In [112]:
#Rename columns
df.columns = df.columns.str.replace(' ', '', regex=False)   #remove whitespaces in column names 
df = df.rename(columns={'StudyHours':'StudyHours(Daily)'})
df.isna().sum()

#***Notes***
#1 - Some records hold student name with null value, in this scenario it's fine to leave null
#EDA does not involve student name, and prediction does not use name as feature

#2 - Null values in 'FinalGrade' column can be used as test target for prediction
#not necessary to remove or inspect 'FinalGrade' yet

StudentID                     0
Name                         29
Gender                        0
AttendanceRate                0
PreviousGrade                 0
ExtracurricularActivities     0
ParentalSupport               0
FinalGrade                   34
StudyHours(Daily)             0
OnlineClassesTaken            0
dtype: int64

In [114]:
#save dataset ***run once***
current_dirr = os.getcwd()
file_name = os.path.join(current_dirr, 'cleaned_dataset.csv')

#check if there is existing file before saving
if not os.path.exists(file_name):
    df.to_csv(file_name, index=False)
    print('File saved successfully.')
else:
    # If the file exists, print a message and do not overwrite
    print('File already exists. Skipping save operation.')

File saved successfully.


Machine Learning Section

In [115]:
#load dataset
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Student Performance Prediction/cleaned_dataset.csv')

#separate train and test dataframes, one with final grade NA, and one with no NA
mask = df['FinalGrade'].isna()
train_df = df[df['FinalGrade'].notna()]
test_df = df[df['FinalGrade'].isna()]

#split into train and test set
X_train = train_df.drop(columns=['StudentID','Name','FinalGrade'])
y_train = train_df['FinalGrade']
X_test = test_df.drop(columns=['StudentID','Name','FinalGrade'])

#convert categorical columns into numeric, match the columns in X_train and X_test
categorical_cols = X_train.select_dtypes(include='object').columns.tolist()
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

#train linear regression model
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
lin_reg_pred = lin_reg.predict(X_test)

#train XG boost mode
xg_reg = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xg_reg.fit(X_train, y_train)
xg_reg_pred = xg_reg.predict(X_test)



In [116]:
#evaluate models
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

#RMSE evaluation
lin_reg.fit(X_train, y_train)
lin_reg_pred = lin_reg.predict(X_test)
print('Linear Regression RMSE:', root_mean_squared_error(y_test, lin_reg_pred))

xg_reg.fit(X_train, y_train)
xgb_reg_pred = xg_reg.predict(X_test)
print('XGBoost RMSE:', root_mean_squared_error(y_test, xgb_reg_pred))

#MAE evaluation
print('\nLinear Regression MAE: ', mean_absolute_error(y_test, lin_reg_pred))
print('XGBoost MAE:', mean_absolute_error( y_test, xgb_reg_pred))

#R2 score evaluation
print('\nLinear Regression R2 Score: ', r2_score(y_test, lin_reg_pred))
print('XGBoost R2 Score:', r2_score( y_test, xgb_reg_pred))

#MAPE evaluation
print('\nLinear Regression MAPE: ', np.mean(np.abs((y_test - lin_reg_pred) / y_test)) * 100)
print('XGBBoost MAPE:', np.mean(np.abs((y_test - xgb_reg_pred) / y_test)) * 100)

#***Notes***
#RMSE result from linear regression shows a better result, therefore Linear Regression model is selected

Linear Regression RMSE: 10.00346357147904
XGBoost RMSE: 12.090825428099425

Linear Regression MAE:  8.610618612921707
XGBoost MAE: 10.180183644675038

Linear Regression R2 Score:  -0.022922307783414198
XGBoost R2 Score: -0.4943549319843532

Linear Regression MAPE:  11.36205925908685
XGBBoost MAPE: 13.294871078204991


In [117]:
#apply median to NA values in finalgrade and re-predict 
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Student Performance Prediction/cleaned_dataset.csv')

X = df.drop(columns=['StudentID','Name','FinalGrade'])
y = df['FinalGrade']

#set new train and test set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
train_median = y_train.median()
y_train = y_train.fillna(train_median)
y_test = y_test.fillna(train_median)
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

#evaluate model
lin_reg.fit(X_train, y_train)
lin_reg_pred = lin_reg.predict(X_test)
print('RMSE:', root_mean_squared_error(y_test, lin_reg_pred))
print('MAE: ', mean_absolute_error(y_test, lin_reg_pred))
print('Score: ', r2_score(y_test, lin_reg_pred))
print('MAPE: ', np.mean(np.abs((y_test - lin_reg_pred) / y_test)) * 100)

#applying median scores improve RMSE result

RMSE: 8.84687523659246
MAE:  7.37683831486669
Score:  0.017794459267692853
MAPE:  9.614702607155744


In [118]:
#log y target prediction
y_train_log = np.log1p(y_train)  

lin_reg.fit(X_train, y_train_log)
xg_reg.fit(X_train, y_train_log)

lin_log_pred = lin_reg.predict(X_test)
xgb_log_pred = xg_reg.predict(X_test)

lin_log_pred = np.expm1(lin_log_pred)
xgb_log_pred = np.expm1(xgb_log_pred)

print('RMSE:', root_mean_squared_error(y_test, lin_log_pred))
print('MAE: ', mean_absolute_error(y_test, lin_log_pred))
print('R2 Score: ', r2_score(y_test, lin_log_pred))
print('MAPE: ', np.mean(np.abs((y_test - lin_log_pred) / y_test)) * 100)

#log y target shows declination in RMSE result 

RMSE: 8.868629436189513
MAE:  7.446978846781056
R2 Score:  0.012958092679918876
MAPE:  9.632205400115097


In [119]:
#outlier treatment
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Student Performance Prediction/cleaned_dataset.csv')

X = df.drop(columns=['StudentID','Name','FinalGrade'])
y = df['FinalGrade']

#set outlier to 5th and 95th caps
lower = y.quantile(0.05)
upper = y.quantile(0.95)
y_clip = y.clip(lower, upper)

#set train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y_clip, test_size=0.2, random_state=42)

#apply median input 
y_train_median = y_train.median()
y_train = y_train.fillna(y_train_median)
y_test = y_test.fillna(y_train_median)

#convert categorical to numeric
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

#check feature importance with standardized feature on prediction
#standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

#fit linear regression
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

#coefficients as feature importance
sig_feature = pd.Series(abs(lin_reg.coef_), index=X_train.columns)
sig_feature = sig_feature.sort_values(ascending=False)
print(sig_feature)

#predict
y_pred = lin_reg.predict(X_test_scaled)

#evaluate
print("\nRMSE:", root_mean_squared_error(y_test, y_pred))
print("MAE :", mean_absolute_error(y_test, y_pred))
print('R2 Score: ', r2_score(y_test, y_pred))
print("MAPE:", np.mean(np.abs((y_test - y_pred) / y_test)) * 100)


#grid gave a worse result,  means features are weak for prediction -
#shrinking coefficients slightly hurt predictions instead of helping

ParentalSupport_Medium       0.377222
StudyHours(Daily)            0.232557
ParentalSupport_Low          0.229212
AttendanceRate               0.198980
OnlineClassesTaken_Yes       0.163287
Gender_Male                  0.121213
ExtracurricularActivities    0.109008
PreviousGrade                0.069795
dtype: float64

RMSE: 8.845895462908386
MAE : 7.37601675875539
R2 Score:  0.018012001807145683
MAPE: 9.60992222482022


In [120]:
#engineering interaction features and drop < 0.1 feature
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Student Performance Prediction/cleaned_dataset.csv')

df['StudyAttendance'] = df['StudyHours(Daily)'] * df['AttendanceRate']
df['StudyLowSupport'] = (df['StudyHours(Daily)'] * (df['ParentalSupport'] == 'Low').astype(int))
df['AttendanceMidSupport'] = (df['AttendanceRate'] * (df['ParentalSupport'] == 'Medium').astype(int))

#readjust features
X = df.drop(columns=['StudentID','Name','FinalGrade','PreviousGrade'])
y = df['FinalGrade']

#set outlier to 5th and 95th caps
lower = y.quantile(0.05)
upper = y.quantile(0.95)
y_clip = y.clip(lower, upper)

#set train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y_clip, test_size=0.2, random_state=42)

#apply median input 
y_train_median = y_train.median()
y_train = y_train.fillna(y_train_median)
y_test = y_test.fillna(y_train_median)

#convert categorical to numeric
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

#standardize the features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#fit linear regression
lin_reg.fit(X_train_scaled, y_train)

#coefficients as feature importance
sig_feature = pd.Series(abs(lin_reg.coef_), index=X_train.columns)
sig_feature = sig_feature.sort_values(ascending=False)
print(sig_feature)

#predict 
y_pred = lin_reg.predict(X_test_scaled)

#evaluate
print("\nRMSE:", root_mean_squared_error(y_test, y_pred))
print("MAE :", mean_absolute_error(y_test, y_pred))
print('R2 Score: ', r2_score(y_test, y_pred))
print("MAPE:", np.mean(np.abs((y_test - y_pred) / y_test)) * 100)

#interactive features implementation increased RMSE due to multicollinearity

#summary 
#predicted final grades compressed as there are only weak signals, all predictors have small coefficients
#clipping outliers also made model learn slightly less variation
#the model predicts near mean grade

ParentalSupport_Medium       4.067883
AttendanceMidSupport         3.709460
StudyAttendance              0.660019
ParentalSupport_Low          0.607077
StudyLowSupport              0.466365
StudyHours(Daily)            0.268387
OnlineClassesTaken_Yes       0.164118
Gender_Male                  0.125798
ExtracurricularActivities    0.110806
AttendanceRate               0.098594
dtype: float64

RMSE: 8.882944549060955
MAE : 7.426344823075087
R2 Score:  0.00976909489253297
MAPE: 9.672971785808084


Final Prediction After Adjustments and Evaluations

In [121]:
#load dataset
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Student Performance Prediction/cleaned_dataset.csv')

#define X and y features
X = df.drop(columns=['StudentID','Name','FinalGrade'])
y = df['FinalGrade']

#set outlier to 5th and 95th caps
lower = y.quantile(0.05)
upper = y.quantile(0.95)
y_clip = y.clip(lower, upper)

#train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y_clip, test_size=0.2, random_state=42)

#apply median input 
y_train_median = y_train.median()
y_train = y_train.fillna(y_train_median)
y_test = y_test.fillna(y_train_median)

#convert categorical to numeric
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

#standardize the features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#fit linear regression
lin_reg.fit(X_train_scaled, y_train)

#predict
y_pred = lin_reg.predict(X_test_scaled)
y_train_pred = lin_reg.predict(X_train_scaled)

#apply predictions
df.loc[X_test.index,'PredictedFinalGrade'] = y_pred
df.loc[X_train.index, 'PredictedFinalGrade'] = y_train_pred
df['PredictedFinalGrade'] = df['PredictedFinalGrade'].apply(lambda x: int(x))

#save predicted final grades
current_dir = os.getcwd()
file_name = os.path.join(current_dir, 'predicted_performance.csv')
if not os.path.exists(file_name):
    df.to_csv(file_name, index=False)
    print('File saved successfully.')
else:
    # If the file exists, print a message and do not overwrite
    print('File already exists. Skipping save operation.')


File saved successfully.


Feature Study and Visualization Section

In [3]:
#load dataset
df = pd.read_csv('C:/Users/josep/OneDrive/Desktop/small python project/Student Performance Prediction/cleaned_dataset.csv')

#fill empty FinalGrade with median
median_grade = df['FinalGrade'].median()
df['FinalGrade'] = df['FinalGrade'].fillna(median_grade)

#convert final grade to 'int'
df['FinalGrade'] = df['FinalGrade'].apply(lambda x: int(x))
df

,StudentID,Name,Gender,AttendanceRate,PreviousGrade,ExtracurricularActivities,ParentalSupport,FinalGrade,StudyHours(Daily),OnlineClassesTaken
0,1,John,Male,85,78,1,High,80,4.8,No
1,2,Sarah,Female,90,85,2,Medium,87,2.2,Yes
2,3,Alex,Male,78,65,0,Low,68,4.6,No
3,4,Michael,Male,92,90,3,High,92,2.9,No
4,5,Emma,Female,85,82,2,Medium,85,4.1,Yes
...,...,...,...,...,...,...,...,...,...,...
842,995,Jordan Thornton,Female,78,85,1,Low,72,1.2,Yes
843,996,Matthew Winters,Male,78,60,0,Medium,72,2.8,No
844,997,Nancy Gibson,Female,78,82,1,Medium,90,2.2,Yes
845,998,Nina Fowler,Male,70,78,3,Medium,68,4.1,Yes


In [10]:
#Study on Attendance Rate and StudyHours(Daily) influence on FinalGrade 
def attendance_study_grade(df):

    temp_df = df.copy()
    temp_df['Above 80'] = temp_df['FinalGrade'].apply(lambda x: 'A' if x >= 80 else 'Not A')
    temp_df = temp_df.groupby('Above 80')['StudentID'].count().reset_index(name='count')
    fig1 = px.bar(temp_df, x='count', y='Above 80', orientation='h', color='Above 80', title='Distribution of Students scoring A and not A',
                  labels={'Above 80':'Score Grade'})
    fig1.show()

    temp_df = df.copy()
    temp_df['Above 80'] = temp_df['FinalGrade'].apply(lambda x: 'A' if x >= 80 else 'Not A')
    fig2 = px.scatter_3d(temp_df, x='AttendanceRate', y='StudyHours(Daily)', z='FinalGrade', color='Above 80', 
                        color_continuous_scale='Viridis', opacity=0.7,
                        hover_data={'Above 80':False, 'AttendanceRate':True,'StudyHours(Daily)':True,'FinalGrade':True},
                        labels={'Above 80':'Score Grade'}, title='Daily StudyHours and AttendanceRate on FinalGrade')
    fig2.update_layout(height=700)
    fig2.show()   

    temp_df = df.copy()
    result_df = pd.DataFrame({
        'HighAttendanceLowStudy': 
        [
            ((temp_df['AttendanceRate'] >= 90) & 
            (temp_df['StudyHours(Daily)'] <= 1) & (temp_df['FinalGrade'] >= 80)).sum()
        ],
        'LowAttendanceHighStudy': 
        [
            ((temp_df['AttendanceRate'] <= 80) & 
            (temp_df['StudyHours(Daily)'] >= 4) & (temp_df['FinalGrade'] >= 80)).sum()
        ],
        'HighAttendanceHighStudy':
        [
            ((temp_df['AttendanceRate'] >= 90) & 
            (temp_df['StudyHours(Daily)'] >= 4) & (temp_df['FinalGrade'] >= 80)).sum()
        ],
        'Final Grade': 'A'
    })

    result2_df = pd.DataFrame({
        'HighAttendanceLowStudy': 
        [
            ((temp_df['AttendanceRate'] >= 90) & 
            (temp_df['StudyHours(Daily)'] <= 1) & (temp_df['FinalGrade'] < 80)).sum()
        ],
        'LowAttendanceHighStudy': 
        [
            ((temp_df['AttendanceRate'] <= 80) & 
            (temp_df['StudyHours(Daily)'] >= 4) & (temp_df['FinalGrade'] < 80)).sum()
        ],
        'HighAttendanceHighStudy':
        [
            ((temp_df['AttendanceRate'] >= 90) & 
            (temp_df['StudyHours(Daily)'] >= 4) & (temp_df['FinalGrade'] < 80)).sum()
        ],
        'Final Grade': 'Not A'
    })

    temp_df = pd.concat([result_df, result2_df], ignore_index=True)
    temp_df = temp_df.melt(id_vars='Final Grade', value_vars=['HighAttendanceLowStudy', 'LowAttendanceHighStudy','HighAttendanceHighStudy'],
                           var_name='Outcome', value_name='count')

    fig3 = px.bar(temp_df, x='Final Grade', y='count', color='Outcome', facet_col='Outcome', text='count',
                  title='Number of Students with Final Grade A by Attendance and Studies')
    fig3.for_each_annotation(lambda x: x.update(text=x.text.replace('Outcome=','Number of ')))
    fig3.update_traces(textfont_size=14, textfont_color='white', textfont_weight='bold')
    fig3.show()

    temp_df = df.copy()
    temp_df['Above 80'] = temp_df['FinalGrade'].apply(lambda x: 'A' if x >= 80 else 'Not A')
    temp_df = temp_df[(temp_df['StudyHours(Daily)']<=1) & (temp_df['AttendanceRate']<=70)]
    temp_df = temp_df.groupby('Above 80')['StudentID'].count().reset_index(name='count')
    print(temp_df)

    #this dataset is slightly skewed towards students scoring A as their final grade
    #graph shows there is not a strong correlation between attendance rate and daily study hours on scoring high for final exam
    #there are students who has good attendance rate but low study hours performing well, and vice versa
    #there are also students who has bad attendance and low study hours but perform bad, and vice versa
    #this proves that other factors do matter
    #the disparity in result in fig3 shows high attendance affects FinalGrade more than high study
    #even among 20 students who have low study hours daily and low attendance rate, 12 of them scored A and 8 not A

attendance_study_grade(df)

  Above 80  count
0        A     12
1    Not A      8


In [158]:
#Study on relationship between parental support and grade improvements (assumptions that parental support helps improve final grade)
def parental_grade(df):

    temp_df = df.copy()
    temp_df['GradeImprovement'] = temp_df['FinalGrade'] - temp_df['PreviousGrade']
    result_df = pd.DataFrame({
        'Improved (>1)': 
        [
            ((temp_df['GradeImprovement'] > 1) & 
            (temp_df['ParentalSupport'] == 'High')).sum()
        ],
        'No Improvement (≤0)': 
        [
            ((temp_df['GradeImprovement'] <= 0) & 
            (temp_df['ParentalSupport'] == 'High')).sum()
        ],
        'Parental Support': 'High'
    })

    result2_df = pd.DataFrame({
        'Improved (>1)': 
        [
            ((temp_df['GradeImprovement'] > 1) & 
            (temp_df['ParentalSupport'] == 'Low')).sum()
        ],
        'No Improvement (≤0)': 
        [
            ((temp_df['GradeImprovement'] <= 0) & 
            (temp_df['ParentalSupport'] == 'Low')).sum()
        ],
        'Parental Support': 'Low'
    })

    temp_df = pd.concat([result_df,result2_df], ignore_index=True)
    temp_df = temp_df.melt(id_vars='Parental Support', value_vars=['Improved (>1)', 'No Improvement (≤0)'],
                           var_name='Outcome', value_name='count')
    
    fig1 = px.bar(temp_df, x='Parental Support', y='count', color='Outcome', facet_col='Outcome', text='count',
                  title='Number of Students with Grade Improvements / Declination by Parental Support')
    fig1.for_each_annotation(lambda x: x.update(text=x.text.replace('Outcome=','Number of ')))
    fig1.update_traces(textfont_size=14, textfont_color='white', textfont_weight='bold')
    fig1.show()

    temp_df = temp_df = df.copy()
    temp_df['GradeImprovement'] = temp_df['FinalGrade'] - temp_df['PreviousGrade']
    temp_df = temp_df[temp_df['ParentalSupport'] != 'Medium']
    temp_df = temp_df.groupby(['ParentalSupport','GradeImprovement'])['StudentID'].count().reset_index(name='count')
    temp_df = temp_df.sort_values(by='GradeImprovement', ascending=True)

    fig2 = px.line(temp_df, x='GradeImprovement', y='count', color='ParentalSupport', markers=True,
                    title='No. of Students with Grade Improvement with "High"/"Low" Parental Support',
                    labels={'count':'No.of Students','GradeImprovement':'Score Difference (Previous vs Final)'})
    
    for i, support in enumerate(temp_df['ParentalSupport'].unique()):
        neg_data = temp_df[(temp_df['ParentalSupport'] == support) & (temp_df['GradeImprovement'] < 0)]
        if not neg_data.empty:
            neg_max = neg_data.loc[neg_data['count'].idxmax()]
            fig2.add_annotation(
                x=neg_max['GradeImprovement'],
                y=neg_max['count'],
                text=f'{support}: {neg_max['count']}',
                showarrow=True,
                arrowhead=2,
                ax=-40,
                ay=-30 - (i*10),
                bordercolor='red',
                borderwidth=1,
                borderpad=4,
                bgcolor='red',
                font_color='white', 
                font_weight='bold'
            )

        pos_data = temp_df[(temp_df['ParentalSupport'] == support) & (temp_df['GradeImprovement'] > 0)]
        if not pos_data.empty:
            pos_max = pos_data.loc[pos_data['count'].idxmax()]
            fig2.add_annotation(
                x=pos_max['GradeImprovement'],
                y=pos_max['count'],
                text=f'{support}: {pos_max['count']}',
                showarrow=True,
                arrowhead=2,
                ax=40,
                ay=-30 + (i*30),
                bordercolor='green',
                borderwidth=1,
                borderpad=4,
                bgcolor='green',
                font_color='white', 
                font_weight='bold'
            )

    fig2.show()

    #Overall the number of students with grade improvements with 'High' parental support is more than 'Low' Parental support
    #this study show parental support has impact on grade improvements
    #however, the number of grade impromements is not heavily influenced by parental support
    #the highest number of grade improvments (22) are recorded at 2 from previous grade, which is not significant
    #the highest number of grade decline (19) are recorded at -10 from previous grade for 'High' parental support
    #assumption : there may be some students who perform worse or lose dmotivation with parental support, whereas some students excel with parental support

    '''
    neg_max = temp_df[temp_df['GradeImprovement'] < 0].loc[temp_df[temp_df['GradeImprovement'] < 0]['count'].idxmax()]
    pos_max = temp_df[temp_df['GradeImprovement'] > 0].loc[temp_df[temp_df['GradeImprovement'] > 0]['count'].idxmax()]
    fig2.add_annotation(x=neg_max['GradeImprovement'], y=neg_max['count'], text=f'Highest: {neg_max["count"]}',
                        showarrow=True, arrowhead=2, ax=0, ay=-20, font_color='red')
    fig2.add_annotation(x=pos_max['GradeImprovement'], y=pos_max['count'], text=f'Highest: {pos_max["count"]}',
                        showarrow=True, arrowhead=2, ax=0, ay=-20, font_color='green')
    '''

parental_grade(df)


In [149]:
#Study on impact of online class taken on grade improvements
def onlineclass_grade(df):
    
    temp_df = df.copy()
    temp_df['GradeImprovement'] = temp_df['FinalGrade'] - temp_df['PreviousGrade']

    result_df = pd.DataFrame({
        'Improved (>1)':
        [
            ((temp_df['GradeImprovement'] > 0) & 
             (temp_df['OnlineClassesTaken'] == 'Yes')).sum()
        ],
        'No Improvement (≤0)':
        [
            ((temp_df['GradeImprovement'] <= 0 ) &
             (temp_df['OnlineClassesTaken'] == 'Yes')).sum()
        ],
        'OnlineClassesTaken':'Yes'
    })

    result2_df = pd.DataFrame({
        'Improved (>1)':
        [
            ((temp_df['GradeImprovement'] > 0) & 
             (temp_df['OnlineClassesTaken'] == 'No')).sum()
        ],
        'No Improvement (≤0)':
        [
            ((temp_df['GradeImprovement'] <= 0 ) &
             (temp_df['OnlineClassesTaken'] == 'No')).sum()
        ],
        'OnlineClassesTaken':'No'
    })

    temp_df = pd.concat([result_df,result2_df], ignore_index=True)
    temp_df = temp_df.melt(id_vars='OnlineClassesTaken', value_vars=['Improved (>1)', 'No Improvement (≤0)'],
                           var_name='Outcome', value_name='count')
    
    fig1 = px.bar(temp_df, x='OnlineClassesTaken', y='count', color='Outcome', facet_col='Outcome', text='count',
                  title='Number of Students with Grade Improvements / Declination by Online Class Taken')
    fig1.for_each_annotation(lambda x: x.update(text=x.text.replace('Outcome=','Number of ')))
    fig1.update_traces(textfont_size=14, textfont_color='white', textfont_weight='bold')
    fig1.show()

    temp_df = temp_df = df.copy()
    temp_df['GradeImprovement'] = temp_df['FinalGrade'] - temp_df['PreviousGrade']
    temp_df = temp_df.groupby(['OnlineClassesTaken','GradeImprovement'])['StudentID'].count().reset_index(name='count')
    temp_df = temp_df.sort_values(by='GradeImprovement', ascending=True)

    fig2 = px.line(temp_df, x='GradeImprovement', y='count', color='OnlineClassesTaken', markers=True,
                    title='No. of Students with Grade Improvement with "Yes"/"No" Online Classes',
                    labels={'count':'No.of Students','GradeImprovement':'Score Difference (Previous vs Final)'})
    
    for i, status in enumerate(temp_df['OnlineClassesTaken'].unique()):
        neg_data = temp_df[(temp_df['OnlineClassesTaken'] == status) & (temp_df['GradeImprovement'] < 0)]
        if not neg_data.empty:
            neg_max = neg_data.loc[neg_data['count'].idxmax()]
            fig2.add_annotation(
                x=neg_max['GradeImprovement'],
                y=neg_max['count'],
                text=f'{status}: {neg_max['count']}',
                showarrow=True,
                arrowhead=2,
                ax=0,
                ay=-30 - (i*20),
                bordercolor='red',
                borderwidth=1,
                borderpad=4,
                bgcolor='red',
                font_color='white', 
                font_weight='bold'
            )

        pos_data = temp_df[(temp_df['OnlineClassesTaken'] == status) & (temp_df['GradeImprovement'] > 0)]
        if not pos_data.empty:
            pos_max = pos_data.loc[pos_data['count'].idxmax()]
            fig2.add_annotation(
                x=pos_max['GradeImprovement'],
                y=pos_max['count'],
                text=f'{status}: {pos_max['count']}',
                showarrow=True,
                arrowhead=2,
                ax=50,
                ay=-30 + (i*1),
                bordercolor='green',
                borderwidth=1,
                borderpad=4,
                bgcolor='green',
                font_color='white', 
                font_weight='bold'
            )

    fig2.show()

    #online classes taken 'Yes' show a more improvement in grade than 'No', and 'No' show larger number of students with no improvement
    #online classes taken does influence grade improvements to much bigger extent than parental support


onlineclass_grade(df)

In [11]:
#study on extracurricular activities influence on final grade
def curricular_grade(df):

    temp_df = df.copy()
    temp_df['Above 80'] = temp_df['FinalGrade'].apply(lambda x: 'A' if x >= 80 else 'Not A')
    temp_df = temp_df.groupby(['ExtracurricularActivities','Above 80'])['FinalGrade'].mean().reset_index(name='avg')
    temp_df['avg'] = temp_df['avg'].apply(lambda x: round(x,2))

    colors = {'Not A':'red','A':'green'}
    fig1 = px.bar(temp_df, x='ExtracurricularActivities', y='avg', color='Above 80', barmode='group', text='avg', color_discrete_map=colors,
                   title='Average Final Grade by ExtracurricularActivities', 
                   labels={'ExtracurricularActivities':'Extracurricular Activities','avg':'Average Score','Above 80':'Score Grade'})
    fig1.update_traces(textfont_size=14, textfont_color='white', textfont_weight='bold')
    fig1.show()

    temp_df = df.copy()
    temp_df = temp_df.groupby('ExtracurricularActivities')['FinalGrade'].agg(Highest='max', Lowest='min').reset_index(names='ExtracurricularActivities')
    max_counts = (df.merge(temp_df, on='ExtracurricularActivities').query('FinalGrade == Highest')
                  .groupby('ExtracurricularActivities')['StudentID'].count().reset_index(name='max_count'))
    min_counts = (df.merge(temp_df, on='ExtracurricularActivities').query('FinalGrade == Lowest')
                  .groupby('ExtracurricularActivities')['StudentID'].count().reset_index(name='min_count'))
    
    temp_df = max_counts.merge(min_counts, on='ExtracurricularActivities')
    temp_df = temp_df.melt(id_vars='ExtracurricularActivities', value_vars=['min_count','max_count'], var_name='type', value_name='count')
    temp_df = temp_df.sort_values(by='ExtracurricularActivities')
    
    fig2 = px.bar(temp_df, x='ExtracurricularActivities', y='count', color='type', barmode='group', text='count',
                  title='No.of Students Achieving Max and Min FinalGrade based on Extracurricular Activities',
                  labels={'ExtracurricularActivities':'Extracurricular Activities','avg':'Average Score','count':'No.of Students'})
    fig2.update_traces(textfont_size=14, textfont_color='white', textfont_weight='bold')
    fig2.show()

    colors = {'min_count':'darkred','max_count':'purple'}
    fig3 = px.line(temp_df, x='ExtracurricularActivities', y='count', color='type', markers=True, color_discrete_map=colors,
                   title='No.of Students Achieving Max and Min FinalGrade Trend by Extracurricular Activities',
                   labels={'ExtracurricularActivities':'Extracurricular Activities','avg':'Average Score','count':'No.of Students'})
    
    min_data = temp_df[temp_df['type'] == 'min_count']
    if not min_data.empty:
        min_max = min_data.loc[min_data['count'].idxmax()]
        min_min = min_data.loc[min_data['count'].idxmin()]
        fig3.add_annotation(
            x=min_max['ExtracurricularActivities'],
            y=min_max['count'],
            text=f'Min Count: {min_max["count"]}',
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-40,
            bordercolor='red',
            borderwidth=1,
            borderpad=4,
            bgcolor='red',
            font_color='white',
            font_weight='bold'
        )
        fig3.add_annotation(
            x=min_min['ExtracurricularActivities'],
            y=min_min['count'],
            text=f'Min Count: {min_min["count"]}',
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-30,
            bordercolor='red',
            borderwidth=1,
            borderpad=4,
            bgcolor='red',
            font_color='white',
            font_weight='bold'
        )
    max_data = temp_df[temp_df['type'] == 'max_count']
    if not max_data.empty:
        max_max = max_data.loc[max_data['count'].idxmax()]
        min_max = max_data.loc[max_data['count'].idxmin()]
        fig3.add_annotation(
            x=max_max['ExtracurricularActivities'],
            y=max_max['count'],
            text=f'Max Count: {max_max["count"]}',
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-30,
            bordercolor='green',
            borderwidth=1,
            borderpad=4,
            bgcolor='green',
            font_color='white',
            font_weight='bold'
        )
        fig3.add_annotation(
            x=min_max['ExtracurricularActivities'],
            y=min_max['count'],
            text=f'Max Count: {min_max["count"]}',
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-70,
            bordercolor='green',
            borderwidth=1,
            borderpad=4,
            bgcolor='green',
            font_color='white',
            font_weight='bold'
        )
        fig3.add_vrect(x0=-0.1, x1=2.1, line_width=0, fillcolor="green", opacity=0.12)
        fig3.update_traces(marker_size=9)

    fig3.show()

    #extracurricular activities has almost zero influence on average final grade obtained by students
    #all number of activities show same max and min score (92 and 62)
    #the biggest difference between number of students scoring lowest and highest grade appeared in most number of extracurricular activity
    #the difference shows extracurricular activities may actually have positive impact among high achievers 
    #most 92 score students had 2 extracurricular activities and most 62 score students had 1 extracurricular
    #there is a small trend increase of number of students achieving 92 score with increase of activities from 0 to 2
    #similarly, there is a decrease trend for number of stduents achieving 62 scores with activities from 1 to 3
    #assumption : extracurricular activities can be a form of stress relief that help high achieving and low achieving students concerntrate more during studies
    #             thus, being able to score better for final exam, however, too many activities can hinder high achieving students

curricular_grade(df)


In [12]:
#other general graphs
def other_grade(df):

    temp_df = df.copy()
    fig1 = px.violin(temp_df, x='Gender', y='FinalGrade', color='Gender', box=True, title='Final Grade Distribution by Gender', points='all')
    fig1.show()

    #dataset is slightly skewed towards higher grade for both Male and Female
    #no to minimal outliers due to outlier treatments
    #distribution for final grade is identical in both genders

    temp_df = df.copy()
    temp_df['Above 80'] = temp_df['FinalGrade'].apply(lambda x: 'A' if x >= 80 else 'Not A')

    counts = []
    for grade in ['A', 'Not A']:

        group = temp_df[temp_df['Above 80'] == grade]
        row = {'Above 80': grade}

        for col in ['AttendanceRate', 'StudyHours(Daily)', 'ExtracurricularActivities']:
            mean_val = group[col].mean()
            row[col] = group[col].between(mean_val - 1, mean_val + 1).sum()

        row['ParentalSupport'] = (group['ParentalSupport'] == 'High').sum()
        row['OnlineClassesTaken'] = (group['OnlineClassesTaken'] == 'Yes').sum()

        counts.append(row)

    temp_df = pd.DataFrame(counts)
    temp_df = temp_df.melt(id_vars='Above 80', var_name='Feature', value_name='Count')


    fig2 = px.line_polar(temp_df, r='Count', theta='Feature', color='Above 80', line_close=True, 
                         markers=True, title='Consistency of Features for Grade A and Not A')

    fig2.update_layout(polar=dict(radialaxis=dict(visible=False)), legend_title_text='Grade Category')
    fig2.show()

    #identify in which factors do students behave similarly, and how strongly is that behavior associated with getting an A
    #students share a lot of similarities in terms of online classes taken and extracurricular activities
    #students share moderate similarities in terms of parental support and daily study hours
    #students do not share similarities in terms of attendance rate
    #as the distribution of A is more than Not A, it is hard to conclude if these features linearly impact final grade
    #however, based on this graph online class taken and extracuriccular activities features are the features that strongly associate with getting A

    temp_df = df.copy()
    temp_df['StudyHours'] = temp_df['StudyHours(Daily)'].round(0).astype(int)
    study_df = temp_df.groupby(['StudyHours', 'FinalGrade']).size().reset_index(name='count')
    attend_df = temp_df.groupby(['AttendanceRate', 'FinalGrade']).size().reset_index(name='count')
    study_df['Above 80'] = study_df['FinalGrade'].apply(lambda x: 'A' if x >= 80 else 'Not A')
    attend_df['Above 80'] = attend_df['FinalGrade'].apply(lambda x: 'A' if x >= 80 else 'Not A')

    fig3 = px.scatter(study_df, x='StudyHours', y='FinalGrade', size='count', color='Above 80', size_max=40,
                      title='Final Grade Distribution by Study Hours')

    fig4 = px.scatter(attend_df, x='AttendanceRate', y='FinalGrade', size='count', color='Above 80', size_max=40,
                      title='Final Grade Distribution by Attendace Rate')

    fig3.show()
    fig4.show()

    #most students in this dataset have good attendance rate, and a little more varied in terms of study hours (still leaning towards more hours)
    #based on earlier studies, study hours do not really reflect much towards final grade
    #for students with lower grades -> can find ways to improve students study hours by implementing fun study ideas, 
    #                                  or increase extracurricular activities based on earlier studies
    #for students with average to good grades -> find ways to retain study hours, or ways to improve grades without changing study hours

other_grade(df)